# Test Old Years

In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
# Standard libraries
import logging
import os
import pathlib
import sys

# 3rd party libraries
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import seaborn as sns
import sqlalchemy as sa
from functools import reduce

# Local libraries
import pudl
import pudl.constants as pc

In [17]:
logger=logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

In [18]:
pudl_settings = pudl.workspace.setup.get_defaults()
pudl_in = pathlib.Path(pudl_settings['pudl_in'])
ds = pudl.workspace.datastore.Datastore(pudl_in, sandbox=True)
#eia861_raw_dfs = pudl.extract.eia861.Extractor(ds).extract([2019])

## Toggle Settings

In [22]:
eia_inputs = {
    "eia860_years": [2008, 2009],
    "eia860_tables": pudl.constants.pudl_tables["eia860"],
    "eia861_years": [],
    "eia861_tables": pudl.constants.pudl_tables["eia861"],
    "eia923_years": [],
    "eia923_tables": pudl.constants.pudl_tables["eia923"],
}

eia860_tables = eia_inputs["eia860_tables"]
eia860_years = eia_inputs["eia860_years"]
eia861_tables = eia_inputs["eia861_tables"]
eia861_years = eia_inputs["eia861_years"]
eia923_tables = eia_inputs["eia923_tables"]
eia923_years = eia_inputs["eia923_years"]

In [23]:
# Extract EIA forms 923, 860
eia860_raw_dfs = pudl.extract.eia860.Extractor(ds).extract(eia860_years)
#eia861_raw_dfs = pudl.extract.eia861.Extractor(ds).extract(eia861_years)
#eia923_raw_dfs = pudl.extract.eia923.Extractor(ds).extract(eia923_years)

Extracting eia860 spreadsheet data.
Columns for boiler_generator_assn are off: should be 4 but got 6
Columns for generator are off: should be 48 but got 50
Columns for generator_existing are off: should be 76 but got 77
Columns for generator_proposed are off: should be 55 but got 56
Columns for generator_retired are off: should be 75 but got 76
Columns for ownership are off: should be 14 but got 17
Columns for plant are off: should be 45 but got 50
Columns for utility are off: should be 20 but got 21


In [7]:
eia860_raw_dfs.keys()

dict_keys(['boiler_generator_assn', 'generator', 'generator_existing', 'generator_proposed', 'generator_retired', 'ownership', 'plant', 'utility'])

In [25]:
# Transform EIA forms 860, 861, 923
eia860_transformed_dfs = pudl.transform.eia860.transform(eia860_raw_dfs, eia860_tables=eia860_tables)
#eia861_transformed_dfs = pudl.transform.eia861.transform(eia861_raw_dfs, eia861_tables=eia861_tables)
#eia923_transformed_dfs = pudl.transform.eia923.transform(eia923_raw_dfs, eia923_tables=eia923_tables)

Transforming raw EIA 860 DataFrames for ownership_eia860 concatenated across all years.
Transforming raw EIA 860 DataFrames for generators_eia860 concatenated across all years.
Transforming raw EIA 860 DataFrames for plants_eia860 concatenated across all years.
Transforming raw EIA 860 DataFrames for boiler_generator_assn_eia860 concatenated across all years.
Transforming raw EIA 860 DataFrames for utilities_eia860 concatenated across all years.


In [27]:
# create an eia transformed dfs dictionary
eia_transformed_dfs = eia860_transformed_dfs.copy()
#eia_transformed_dfs.update(eia861_transformed_dfs.copy())
#eia_transformed_dfs.update(eia923_transformed_dfs.copy())

In [28]:
# convert types..
eia_transformed_dfs = pudl.helpers.convert_dfs_dict_dtypes(eia_transformed_dfs, 'eia')

/Users/aesharpe/miniconda3/envs/pudl-dev/lib/python3.8/site-packages/pandas/core/missing.py:49: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask = arr == x


,associated_combined_heat_power,bypass_heat_recovery,capacity_mw,carbon_capture,cofire_fuels,county,deliver_power_transgrid,duct_burners,energy_source_code_1,energy_source_code_2,...,original_planned_operating_date,other_modifications_date,planned_derate_date,planned_repower_date,planned_retirement_date,planned_uprate_date,retirement_date,uprate_derate_completed_date,fuel_type_code_pudl,report_date
0,False,<NA>,191.2,<NA>,<NA>,<NA>,<NA>,False,NG,<NA>,...,2002-06-01,NaT,NaT,NaT,NaT,NaT,NaT,NaT,gas,2008-01-01
1,False,<NA>,191.2,<NA>,<NA>,<NA>,<NA>,False,NG,<NA>,...,2002-06-01,NaT,NaT,NaT,NaT,NaT,NaT,NaT,gas,2008-01-01
2,False,<NA>,191.2,<NA>,<NA>,<NA>,<NA>,False,NG,<NA>,...,2002-06-01,NaT,NaT,NaT,NaT,NaT,NaT,NaT,gas,2008-01-01
3,False,<NA>,180.0,<NA>,<NA>,<NA>,<NA>,False,NG,<NA>,...,2004-06-01,NaT,NaT,NaT,NaT,NaT,NaT,NaT,gas,2008-01-01
4,False,<NA>,180.0,<NA>,<NA>,<NA>,<NA>,False,NG,<NA>,...,2004-06-01,NaT,NaT,NaT,NaT,NaT,NaT,NaT,gas,2008-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19553,False,<NA>,0.8,<NA>,<NA>,<NA>,True,False,WAT,<NA>,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,hydro,2008-01-01
19554,True,<NA>,4.0,<NA>,<NA>,<NA>,True,False,WDS,<NA>,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,waste,2008-01-01
19555,False,<NA>,2.8,<NA>,<NA>,<NA>,True,<NA>,NG,DFO,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,gas,2008-01-01
19556,False,<NA>,2.8,<NA>,<NA>,<NA>,True,<NA>,NG,DFO,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,gas,2008-01-01
